In [ ]:
import matplotlib.pylab as plt
from mpl_toolkits import mplot3d
import numpy as np
import pandas as pd
import seaborn as sns

%matplotlib

#### TAN = c(NH$_3$ + c(NH$_4^+$)

sensor response is sigmoidal. It mainly depends on the response time 

In [2]:
# .............................................................
def sensor_response(t, t_resp, conc, function='v1'):
    # sigmoidal sensor response 
    if function == 'v1':
        dfSig = pd.DataFrame(1 / (1 + np.exp((-t+10)*1/t_resp)-0.5) * conc, index=t, 
                             columns=['signal'])
    elif function == 'v2': # y = x / (sqrt(c + x^2))
        dfSig = pd.DataFrame(t*conc / np.sqrt(t_resp + t**2), index=t, 
                             columns=['signal'])
    elif function == 'v3': # deline y = x / (sqrt(c + x^2))
        dfSig = pd.DataFrame((-1*t / np.sqrt(t_resp + t**2)+1)*conc, index=t, 
                             columns=['signal'])
        
    t10 = dfSig[dfSig['signal'] >= dfSig['signal'].max()*0.1].index[0]
    t90 = dfSig[dfSig['signal'] >= dfSig['signal'].max()*0.9].index[0]
    return dfSig, t10, t90

# .............................................................
c_nh3 = 1.5
step = 0.25
t = np.linspace(0, 250, num=int(100/step+1))
dfSig1, t1_10, t1_90 = sensor_response(t=t, t_resp=1., conc=c_nh3, function='v2')
dfSig2, t2_10, t2_90 = sensor_response(t=t, t_resp=50., conc=c_nh3, function='v2')
dfSig3, t3_10, t3_90 = sensor_response(t=t, t_resp=50., conc=c_nh3, function='v3')


# plot sensor response
fig, ax = plt.subplots()
ax.set_title('Sensor response', loc='left')

# sensor 1
ax.plot(t, dfSig1, 'k', label='fast response')
ax.axhline(dfSig1['signal'].max()*0.1, lw=0.5, ls=':', color='darkorange'), 
ax.axhline(dfSig1['signal'].max()*0.9, lw=0.5, ls=':', color='darkorange')

# sensor 2
ax.plot(t, dfSig2, '-.k', label='slow response')
ax.axhline(dfSig2['signal'].max()*0.1, lw=0.5, ls=':', color='forestgreen'), 
ax.axhline(dfSig2['signal'].max()*0.9, lw=0.5, ls=':', color='forestgreen')

# sensor 3
ax.plot(t, dfSig3, ':k', label='declining response')
ax.axhline(dfSig3['signal'].max()*0.9, lw=0.5, ls=':', color='forestgreen'), 
ax.axhline(dfSig3['signal'].max()*0.1, lw=0.5, ls=':', color='forestgreen')


ax.set_xlabel('Time / s'), ax.set_ylabel('Signal / V')
plt.xlim(-5, 155)
sns.despine()
plt.legend(loc='lower right')

sensor signal depends on concentration as well

In [3]:
# def ist_concentration(t_period, conc_0, conc_max):
def target_concentration(c_nh3, tstart, tstop, nP=None, N=None, D=None):
    """
    N = 100 # sample count; a multitude of the time period given
    nP = 2  # frequency; number of cycles/period
    D = 25  # width of pulse; usually half of the period P
    c_nh3   # maximal ammonia concentration
    tstart, tstop # time period given
    """
    if N is None:
        N = tstop - tstart
    if nP is None:
        nP = 1
    P = N/nP
    if D is None:
        D = P/2

    sig = (np.arange(N) % P < D) * c_nh3
    trange = np.linspace(tstart, tstop, num=int(N))
    dfconc = pd.DataFrame(sig, columns=['signal'], index=trange)
    dfconc.index.name='time/s'
    
    return dfconc, D, P, trange

# .................................
start_time, end_time = 0, 60
nsample, nper = 10*(end_time-start_time), 3

dfconc1, D1, P1, t1 = target_concentration(c_nh3=c_nh3, tstart=start_time, tstop=end_time)
dfconc2, D2, P2, t2 = target_concentration(N=nsample, nP=nper, c_nh3=c_nh3, 
                                       tstart=start_time, tstop=end_time)


fig1, ax1 = plt.subplots(figsize=(7,3))
ax1.set_xlabel('Time / s'), ax1.set_ylabel('Analyte concentration')
ax1.plot(dfconc1, color='k', lw=1., marker='.', fillstyle='none', 
         label='N:{:.1f}, period: {:.2f}, freq: 1'.format(end_time-start_time, P1))
ax1.plot(dfconc2, color='darkorange', marker='.', fillstyle='none', lw=0.75,
         label='N:{:.1f}, period: {:.2f}, freq: {:.0f}'.format(nsample, P1, nper))
ax1.legend(loc=0), ax1.set_ylim(-0.05, 1.6)
plt.tight_layout(), sns.despine()
plt.show()

time to combine

signal ~ concentration(t) * response time * some other factors

In [127]:
# gloabl parameter
c_nh3 = 1.5                       # maximal ammonia concentration 
# concentration fluctuation
tstart, tend, step = 0, 100, 0.05 # sec; time range of fluctuation
nP = 2.                           # number of cycles per period
# sensor settings
tstart_sens = 0                   # sec; time range of sensor response            
t_resp = 1.                       # sensor response time

# .............................................
# target concentration
dfconc, D, P, trange = target_concentration(nP=nP, c_nh3=c_nh3, tstart=tstart, tstop=tend)

# .............................................
# sampling rate when a measurement point is taken
s = 0
for en, i in enumerate(trange):
    if i > D:
        if s == 0:
            s = en
sampling_rate = trange[s]
tsampling = np.arange(tstart, tend+sampling_rate, sampling_rate)

# sensor response
sens_time0 = np.linspace(tstart_sens, sampling_rate, num=int((sampling_rate-tstart_sens)/step+1))

# ........................................................................................
# plotting 
# ........................................................................................
fs = 10
fig, ax = plt.subplots(figsize=(6,3))
fig.canvas.set_window_title('sampling rate slower than response time (optimal)')
ax.set_xlabel('Time / s', fontsize=fs), ax.set_ylabel('Analyte concentration', fontsize=fs)
ax.set_ylim(-0.05, 2.)

# target concentration
conc_fluc = ax.plot(dfconc, ':k', label='target concentration')

# sensor response
# each time I change the concentration, the sensor has to respond to the change
# 1) sensor is delayed according to the sampling rate
# 2) response does not equal target concentration as sensor has a response time
ddata = dict()
for t in tsampling: # sensor sampling rate 
    # closest timestep in concentration fluctuation
    tc = min(dfconc.index, key=lambda x:abs(x-t))
    # find position in list for previous concentration
    pos = [en for en in range(len(dfconc.index)) if dfconc.index[en] == tc][0]
    tc_1 = dfconc.index[pos-1]
    
    # update actual sensor measurement time +t
    sens_time = np.linspace(tc, sampling_rate+tc, num=int((sampling_rate-tstart_sens)/step+1))
    # sensor signal response
    if dfconc.loc[tc, 'signal'] == c_nh3 and dfconc.loc[tc_1, 'signal'] == 0:
        df = sensor_response(t=sens_time0, t_resp=t_resp, conc=dfconc.loc[tc, 'signal'], 
                             function='v2')[0] 
    elif dfconc.loc[tc, 'signal'] == 0 and dfconc.loc[tc_1, 'signal'] == c_nh3:
        df = sensor_response(t=sens_time0, t_resp=t_resp, conc=c_nh3, function='v3')[0] 
    elif dfconc.loc[tc, 'signal'] == 0 and dfconc.loc[tc_1, 'signal'] == 0:
        df = sensor_response(t=sens_time0, t_resp=t_resp, conc=0, function='v3')[0]
    elif dfconc.loc[tc, 'signal'] == c_nh3 and dfconc.loc[tc_1, 'signal'] == c_nh3:
        print('track 4')
    ddata[tc] = df
    ax.plot(sens_time, df, lw=1, color='navy', label='sensor response')
    
    # arrows indicating sampling points
    ax.arrow(tc, 1.9, 0.0, -0.2, fc="k", ec="k", head_width=0.5, head_length=0.1)
ax.legend(['target concentration', 'sensor response'], loc=0, fontsize=fs*0.7)    
plt.tight_layout(), sns.despine()
plt.grid(False)
plt.show()

In [142]:
60*60

3600

In [150]:
# gloabl parameter
c_nh3 = 1.5                       # maximal ammonia concentration 
# concentration fluctuation
tstart, tend, step = 0, 60*5, 0.05 # sec; time range of fluctuation
nP = 5.5                          # number of cycles per period
# sensor settings
tstart_sens = 0                   # sec; start time range of sensor response            
t_resp = 100.                       # sensor response time

# .............................................
# target concentration
dfconc, D, P, trange = target_concentration(nP=nP, c_nh3=c_nh3, tstart=tstart, tstop=tend)

# .............................................
# period (in s) when a measurement point is taken
s = 0
for en, i in enumerate(trange):
    if i > D:
        if s == 0:
            s = en
sampling_rate = trange[s]
tsampling = np.arange(tstart, tend+sampling_rate, sampling_rate)

# sensor response
sens_time0 = np.linspace(tstart_sens, sampling_rate, num=int((sampling_rate-tstart_sens)/step+1))

# ........................................................................................
# plotting 
# ........................................................................................
fs = 10
fig, ax = plt.subplots(figsize=(6,3))
fig.canvas.set_window_title('sampling rate faster than response time (sub-optimal)')
ax.set_xlabel('Time / s', fontsize=fs), ax.set_ylabel('Analyte concentration', fontsize=fs)
ax.set_ylim(-0.05, 2.2)

# target concentration
conc_fluc = ax.plot(dfconc, ':k', label='target concentration')

# sensor response
# each time I change the concentration, the sensor has to respond to the change
# 1) sensor is delayed according to the sampling rate
# 2) response does not equal target concentration as sensor has a response time
ddata, c_apparent = dict(), 0
for t in tsampling: # sensor sampling rate 
    # closest timestep in concentration fluctuation
    tc = min(dfconc.index, key=lambda x:abs(x-t))

    if tc == dfconc.index[-1]:
        pass
    else:
        # find position in list for previous concentration
        pos = [en for en in range(len(dfconc.index)) if dfconc.index[en] == tc][0]
        tc_1 = dfconc.index[pos-1]

        # update actual sensor measurement time +t
        sens_time = np.linspace(tc, sampling_rate+tc, num=int((sampling_rate-tstart_sens)/step+1))
        # sensor signal response
        if dfconc.loc[tc, 'signal'] == c_nh3 and dfconc.loc[tc_1, 'signal'] == 0:
            df = sensor_response(t=sens_time0, t_resp=t_resp, conc=dfconc.loc[tc, 'signal'], 
                                 function='v2')[0] + c_apparent
        elif dfconc.loc[tc, 'signal'] == 0 and dfconc.loc[tc_1, 'signal'] == c_nh3:
            df = sensor_response(t=sens_time0, t_resp=t_resp, conc=c_apparent, 
                                 function='v3')[0] 

        elif dfconc.loc[tc, 'signal'] == 0 and dfconc.loc[tc_1, 'signal'] == 0:
            df = sensor_response(t=sens_time0, t_resp=t_resp, conc=c_apparent,
                                 function='v3')[0] # + c_apparent

        elif dfconc.loc[tc, 'signal'] == c_nh3 and dfconc.loc[tc_1, 'signal'] == c_nh3:
            df = sensor_response(t=sens_time0, t_resp=t_resp, conc=dfconc.loc[tc, 'signal'], 
                                 function='v2')[0] + c_apparent
        ddata[tc], c_apparent = df, df['signal'].to_numpy()[-1]
        ax.plot(sens_time, df, lw=1.25, color='teal', label='sensor response')

        # arrows indicating that the sensor starts to react to the surrounding
        ax.arrow(tc, 1.9, 0.0, -0.2, fc="k", ec="k", head_width=tend/150, head_length=0.1)
        # ax.axvline(tc, 0, 0.8, lw=0.5, alpha=0.5, color='k')
    
ax.legend(['target concentration', 'sensor response'], loc=0, fontsize=fs*0.7)    
plt.tight_layout(), sns.despine(), plt.grid(False)
plt.show()